# Results dashboard

Scans `Results/` and produces:
- nicely formatted summary tables
- confusion matrices + ROC curves (test)
- SHAP/LIME plots when explanation outputs exist

In [ ]:
from pathlib import Path
import sys, os

REPO = Path().resolve()
assert (REPO/'qnm_qai.py').exists(), "Run Jupyter from the repository root (folder containing qnm_qai.py)"
print("Repo root:", REPO)
print("Python:", sys.executable)

# Best-effort: ensure Results exists
(Path("Results")).mkdir(exist_ok=True)


In [ ]:
ensure_matplotlib()
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, balanced_accuracy_score

def _safe_div(num, den):
    return float(num) / float(den) if float(den) != 0.0 else float("nan")

def cm_metrics_from_preds(y_true, prob1, threshold=0.5):
    y_true = np.asarray(y_true, dtype=int)
    prob1 = np.asarray(prob1, dtype=float)
    y_pred = (prob1 >= float(threshold)).astype(int)

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()

    sens = _safe_div(tp, tp + fn)
    spec = _safe_div(tn, tn + fp)
    ppv  = _safe_div(tp, tp + fp)
    npv  = _safe_div(tn, tn + fn)

    acc = accuracy_score(y_true, y_pred)
    bal = balanced_accuracy_score(y_true, y_pred)

    # AUC is undefined if only one class is present
    auc = float("nan")
    if len(np.unique(y_true)) == 2:
        auc = roc_auc_score(y_true, prob1)

    return {
        "threshold": float(threshold),
        "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp),
        "sensitivity": sens,
        "specificity": spec,
        "ppv": ppv,
        "npv": npv,
        "accuracy": float(acc),
        "balanced_accuracy": float(bal),
        "auc": float(auc),
    }, cm

def show_confusion_matrix(cm, title="Confusion matrix", labels=("0", "1")):
    import matplotlib.pyplot as plt
    import numpy as np

    cm = np.asarray(cm, dtype=int)
    fig, ax = plt.subplots(figsize=(4.2, 3.6))
    im = ax.imshow(cm)

    ax.set_title(title)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_xticks([0, 1], labels=labels)
    ax.set_yticks([0, 1], labels=labels)

    for (i, j), v in np.ndenumerate(cm):
        ax.text(j, i, str(v), ha="center", va="center")

    fig.tight_layout()
    plt.show()

def show_roc_curve(y_true, prob1, title="ROC curve"):
    import matplotlib.pyplot as plt
    from sklearn.metrics import RocCurveDisplay

    if len(np.unique(y_true)) < 2:
        print("ROC: only one class present in y_true; skipping.")
        return
    RocCurveDisplay.from_predictions(y_true, prob1)
    plt.title(title)
    plt.show()

def ensure_matplotlib():
    try:
        import matplotlib.pyplot as _plt  # noqa: F401
    except Exception:
        # Notebook-safe install
        import sys
        !{sys.executable} -m pip install matplotlib


## Load all summary result CSVs

In [ ]:
from pathlib import Path
import pandas as pd

summaries = sorted(Path("Results").glob("*__results.csv"))
print("Summary CSVs:", len(summaries))
for p in summaries:
    print(" -", p)

dfs = []
for p in summaries:
    df = pd.read_csv(p)
    df.insert(0, "dataset", p.name.replace("__results.csv",""))
    dfs.append(df)

if dfs:
    all_df = pd.concat(dfs, ignore_index=True)
    display(all_df)
else:
    print("No summary CSVs found. Run: bash examples/run_all_examples.sh")


## Confusion matrices for every method (test set)

In [ ]:
from pathlib import Path
import pandas as pd

for dataset_dir in sorted([p for p in Path("Results").iterdir() if p.is_dir()]):
    print("\n=== DATASET:", dataset_dir.name, "===")
    for method_dir in sorted([p for p in dataset_dir.iterdir() if p.is_dir()]):
        pred_path = method_dir/"predictions"/"test.csv"
        if not pred_path.exists():
            continue
        dfp = pd.read_csv(pred_path)
        if "true_label" not in dfp.columns:
            continue

        y = dfp["true_label"].astype(int).to_numpy()
        prob1 = dfp["prob_1"].astype(float).to_numpy()

        metrics, cm = cm_metrics_from_preds(y, prob1, threshold=0.5)
        print(method_dir.name, {k: metrics[k] for k in ["accuracy","auc","balanced_accuracy","sensitivity","specificity"]})
        show_confusion_matrix(cm, title=f"{dataset_dir.name} / {method_dir.name} — test CM (thr=0.5)")
        show_roc_curve(y, prob1, title=f"{dataset_dir.name} / {method_dir.name} — test ROC")


## SHAP + LIME plots (if present)

In [ ]:
from pathlib import Path
import pandas as pd

def plot_shap_summary(shap_csv: Path, top=15, title=None):
    import matplotlib.pyplot as plt
    df = pd.read_csv(shap_csv)
    cand_cols = [c for c in df.columns if c.lower() in ("mean_abs_shap","mean_abs","abs_mean","mean_abs_value")]
    col = cand_cols[0] if cand_cols else df.columns[-1]
    df = df.sort_values(col, ascending=False).head(int(top)).iloc[::-1]
    plt.figure(figsize=(6.5, 4.5))
    plt.barh(df["feature"], df[col])
    plt.title(title or f"SHAP summary — top {top}")
    plt.xlabel(col)
    plt.tight_layout()
    plt.show()

def plot_lime_for_sample(lime_csv: Path, sample_id=None, top=12, title=None):
    import matplotlib.pyplot as plt
    df = pd.read_csv(lime_csv)
    if "sample_id" not in df.columns:
        for c in df.columns:
            if "sample" in c.lower() and "id" in c.lower():
                df = df.rename(columns={c: "sample_id"})
                break
    if sample_id is None:
        sample_id = df["sample_id"].iloc[0]
    d = df[df["sample_id"] == sample_id].copy()
    wcol = "weight" if "weight" in d.columns else d.columns[-1]
    d["absw"] = d[wcol].abs()
    d = d.sort_values("absw", ascending=False).head(int(top)).sort_values(wcol, ascending=True)
    plt.figure(figsize=(6.5, 4.5))
    plt.barh(d["feature"], d[wcol])
    plt.title(title or f"LIME — sample {sample_id}")
    plt.xlabel(wcol)
    plt.tight_layout()
    plt.show()

for dataset_dir in sorted([p for p in Path("Results").iterdir() if p.is_dir()]):
    for method_dir in sorted([p for p in dataset_dir.iterdir() if p.is_dir()]):
        shap_csv = method_dir/"explain"/"shap"/"shap_summary.csv"
        lime_csv = method_dir/"explain"/"lime"/"lime_explanations.csv"
        if shap_csv.exists():
            print("\nSHAP:", dataset_dir.name, method_dir.name)
            display(pd.read_csv(shap_csv).head(10))
            plot_shap_summary(shap_csv, top=15, title=f"{dataset_dir.name}/{method_dir.name}")
        if lime_csv.exists():
            print("\nLIME:", dataset_dir.name, method_dir.name)
            display(pd.read_csv(lime_csv).head(10))
            plot_lime_for_sample(lime_csv, sample_id=None, top=12, title=f"{dataset_dir.name}/{method_dir.name}")
